# Basic Example

Copy of basic_example_command_line.ipynb except using Python Objects. Behavior should be exactly the same.



# Setup

Installing package:

- Download from Github repository and build
- Add OpenAI Key to appropriate location

In [1]:
from tablevault.tablevault import TableVault


NameError: name 'function' is not defined

# General Functionalities

- Every Table Modification is Logged
- Multi-Process Safe -> Locking supports Single Writes and Multiple Reads
- Re-start Safe (as long as operation is logged, the operation will execute function as intended):
- Robust Versioning
- Granualar (Manual) History Tracking
- Inter-Table Linking (Similar to Foriegn Keys)
- Allow both Row and Column Table Updates

# Initial/External Data
- Yaml Files
- Data Source (pdf): OpenAI generated short story. 
- OpenAI API Key

## Example YAML Files

In [ ]:
import yaml
# Table Generation File
rel_path = './test_data/test_data_db/stories/fetch_stories.yaml'

with open(rel_path, "r") as file:
    yaml_data = yaml.safe_load(file)
yaml_data

In [ ]:
# Code Execution File

rel_path = './test_data/test_data_db/llm_storage/upload_openai.yaml'

with open(rel_path, "r") as file:
    yaml_data = yaml.safe_load(file)
yaml_data

In [ ]:
# LLM Execution File (Only support OpenAI Threads for now)

rel_path = './test_data/test_data_db/llm_questions/question_3.yaml'

with open(rel_path, "r") as file:
    yaml_data = yaml.safe_load(file)
yaml_data

# Make Base Folder

In [ ]:
# Command to initialize folder

#!tablevault database -db test_database -r

tablevault = TableVault('test_database', author = 'jinjin', create = True)

In [ ]:
# View current folder structure
import os

def print_directory_tree(root_dir, indent=" ", files = True):
    for item in os.listdir(root_dir):
        path = os.path.join(root_dir, item)
        if os.path.isdir(path):
            print(indent + "📁 " + item)
            print_directory_tree(path, indent + "    ", files)
        else:
            if files:
                print(indent + "📄 " + item)


print_directory_tree(root_dir = 'test_database', indent=" ")

# Make Table Folders

In [ ]:
# Commands to initialize table folders

# ! tablevault setup-table -db test_database -t stories
tablevault.setup_table('stories') 

#! tablevault setup-table -db test_database -t llm_storage
tablevault.setup_table('llm_storage') 

#! tablevault setup-table -db test_database -t llm_questions -m # allow multiple active versions for different versioning testing
tablevault.setup_table('llm_questions', allow_multiple = True) 


In [ ]:
# View Current folder structure

print_directory_tree(root_dir = 'test_database', indent=" ")

In [ ]:
# View example log and metadata files
log_file = 'test_database/metadata/log.txt'
with open(log_file, 'r') as f:
    content = f.read()
    print(content)

In [ ]:
import json
log_file = 'test_database/metadata/tables_multiple.json'
with open(log_file, 'r') as f:
    content = json.load(f)
    print(content)

**Example Active Log**

Shown in text because file deleted once executed.

# Make Table Instances

## First we add the prompts to the data table "prompts" directories

In [ ]:
import shutil

def copy_files_to_table(base_dir, db_dir, table_name):
    org_path = os.path.join(base_dir, table_name)
    new_path = os.path.join(db_dir, table_name)
    new_path = os.path.join(new_path, 'prompts')
    for file in os.listdir(org_path):
        if file.endswith('.yaml'):
            org_path_ = os.path.join(org_path, file)
            new_path_ = os.path.join(new_path, file)
            if os.path.exists(new_path_):
                os.remove(new_path_)
            shutil.copy2(org_path_, new_path_)

yaml_base_dir = './test_data/test_data_db'
db_dire = './test_database'

copy_files_to_table(yaml_base_dir, db_dire, "stories")
copy_files_to_table(yaml_base_dir, db_dire, "llm_storage")
copy_files_to_table(yaml_base_dir, db_dire, "llm_questions")

## Then we make instances with specified prompts

In [ ]:
# Make Temporary (Not executed) Instances

#!tablevault setup-temp -db test_database -t stories -p fetch_stories -gp fetch_stories
tablevault.setup_temp(table_name = 'stories', prompts = ['fetch_stories'], gen_prompt = 'fetch_stories')
#!tablevault setup-temp -db test_database -t llm_storage -p fetch_llm_storage -p upload_openai -gp fetch_llm_storage
tablevault.setup_temp(table_name = 'llm_storage', prompts = ['fetch_llm_storage', 'upload_openai'], gen_prompt = 'fetch_llm_storage')
#!tablevault setup-temp -db test_database -t llm_questions -v version_1 \
#-p fetch_llm_question -p question_1 -p question_2 -p  question_3 -gp fetch_llm_question
tablevault.setup_temp(table_name = 'llm_questions',version = 'version_1', prompts = ['fetch_llm_question', 'question_1', 'question_2', \
                                                             'question_3'], gen_prompt = 'fetch_llm_question')


In [ ]:
# View Current Folder Structure
print_directory_tree(root_dir = 'test_database/stories', indent=" ")

In [ ]:

rel_path = './test_database/stories/TEMP_/prompts/description.yaml'

with open(rel_path, "r") as file:
    yaml_data = yaml.safe_load(file)
yaml_data

# Execute Table Instances

In [ ]:
# Make PDF Table
#! tablevault execute -db test_database -t stories
tablevault.execute(table_name = 'stories')
print_directory_tree(root_dir = 'test_database/stories', indent=" ")

In [ ]:
story_instance_id = tablevault.list_instances(table_name = 'stories')
story_instance_id = story_instance_id[0]


In [ ]:
import pandas as pd
df_path = f'test_database/stories/{story_instance_id}/table.csv'
df = pd.read_csv(df_path)
df

In [ ]:
# Make OpenAI Storage File (upload pdf to OpenAI) (Allow Multiple Threads)
#! tablevault execute -db test_database -t llm_storage
tablevault.execute(table_name = 'llm_storage')


In [ ]:
# View Table
code_instance_id = tablevault.list_instances(table_name = 'llm_storage')
code_instance_id = code_instance_id[0]

df_path = f'test_database/llm_storage/{code_instance_id}/table.csv'
df = pd.read_csv(df_path)
df

In [ ]:
# Make LLM Question Response Table (Allow Multiple Threads)
tablevault.execute(table_name = 'llm_questions', version = 'version_1')

In [ ]:
llm_instance_id = tablevault.list_instances(table_name = 'llm_questions')
llm_instance_id = llm_instance_id[0]

df_path = f'test_database/llm_questions/{llm_instance_id}/table.csv'
pd.set_option("display.max_colwidth", None)

df = pd.read_csv(df_path)
df

# Table Versioning

We automatically do not re-execute tables if there hasn't been any changes. We only execute cases where the rows (samples), columns (prompts), or dependencies (other tables) have changed by default. 

## No Changes

In [ ]:
# Copy Previous Instances
#!tablevault setup-temp -db test_database -t stories -pid {story_instance_id}
tablevault.setup_temp(table_name = 'stories', prev_id = story_instance_id)
#!tablevault setup-temp -db test_database -t llm_storage -pid {code_instance_id}
tablevault.setup_temp(table_name = 'llm_storage', prev_id = code_instance_id)
#!tablevault setup-temp -db test_database -t llm_questions -pid {llm_instance_id} -v version_1
tablevault.setup_temp(table_name = 'llm_questions', version = 'version_1', prev_id = llm_instance_id)

In [ ]:
# Execute new Instances
#! tablevault execute -db test_database -t stories
tablevault.execute(table_name = 'stories')

#! tablevault execute -db test_database -t llm_storage
tablevault.execute(table_name = 'llm_storage')
#! tablevault execute -db test_database -t llm_questions -v version_1
tablevault.execute(table_name = 'llm_questions', version = 'version_1')

In [ ]:
print_directory_tree(root_dir = 'test_database/stories', indent=" ")

## Column Update 

What happens to execution of LLM questions if we change a column? We will test this on a copied database so the original remains clean.

We generate a new prompt question for llm_questions, and create a new instance with that question.

In [ ]:
# function to replace question_1 prompt with question_1a
def replace_prompt(prompt, instance_id, table_name, base_dir = './test_data/test_data_db', db_dir = './test_database_2', prev_prompt= None):
    org_path = os.path.join(base_dir, table_name, prompt)
    new_dir = os.path.join(db_dir, table_name, instance_id, 'prompts')
    new_path = os.path.join(new_dir, prompt)
    if prev_prompt != None:
        prev_prompt_path = os.path.join(new_dir, prev_prompt)
        if os.path.exists(prev_prompt_path):
            print('Removed')
            print(prev_prompt_path)
            os.remove(prev_prompt_path)
    shutil.copy2(org_path, new_path)

In [ ]:
# Copy database
def copy_db(new_dir = 'test_database_2', orig_dir = 'test_database'):
    if os.path.exists(new_dir):
        shutil.rmtree(new_dir)
    shutil.copytree(orig_dir, new_dir)

copy_db() 
# since everything is saved in the directory, 
# once we copy the data over, we can execute like normal 
# as if we are just continuing previous operations

In [ ]:
# Create new TableVault object
tablevault_2 = TableVault(db_dir = 'test_database_2', author = 'jinjin')

In [ ]:
# generate new temporary instance and replace prompt
#!tablevault setup-temp -db test_database_2 -t llm_questions -pid {llm_instance_id} -v version_2
tablevault_2.setup_temp(table_name = 'llm_questions', version = 'version_2', prev_id = llm_instance_id)

replace_prompt("question_1a.yaml", "TEMP_version_2", "llm_questions", prev_prompt= "question_1.yaml")

print_directory_tree(root_dir = 'test_database_2/llm_questions/TEMP_version_2', indent=" ") # Note how we have a new prompt (user changed)

In [ ]:
# Execute with new prompt
#!tablevault execute -db test_database_2 -t llm_questions -v version_2
tablevault_2.execute(table_name = 'llm_questions', version= 'version_2')

In [ ]:
# If we compare the two dataframes, note only the question 1's column changed
#llm_instance_ids = !tablevault list-instances -db test_database_2 -t llm_questions
llm_instance_ids = tablevault_2.list_instances(table_name = 'llm_questions')

for instance_id in llm_instance_ids:
    df_path = f'test_database_2/llm_questions/{instance_id}/table.csv'
    df = pd.read_csv(df_path)
    print(instance_id)
    display(df)


## Row Update
What happens to execution of LLM questions if we add a row (pdf file)? We will test this on a copied database so the original remains clean.

We insert a new story in the stories folder. We will then re-execute the stories and the open-ai storage dataframes to seel what happens.

In [ ]:
# Functions to copy
def copy_story(base_dir= './test_data/stories', story_name = 'The_Clockmakers_Secret.pdf'):
    org_path = os.path.join(base_dir, story_name)
    new_name = story_name.split(".")[0] + '_copy.pdf'
    new_path = os.path.join(base_dir, new_name)
    shutil.copy2(org_path, new_path)

# Clean Up
def delete_story(base_dir= './test_data/stories', story_name = 'The_Clockmakers_Secret_copy.pdf'):
    story_path  = os.path.join(base_dir, story_name)
    os.remove(story_path)

In [ ]:
copy_db()
copy_story()

In [ ]:
tablevault_2 = TableVault(db_dir = 'test_database_2', author = 'jinjin')

In [ ]:
# make new table instances
#!tablevault setup-temp -db test_database_2 -t stories -pid {story_instance_id}
tablevault_2.setup_temp(table_name = 'stories', prev_id = story_instance_id)
#!tablevault setup-temp -db test_database_2 -t llm_storage -pid {code_instance_id}
tablevault_2.setup_temp(table_name = 'llm_storage', prev_id = code_instance_id)

In [ ]:
# Execute New instances
#! tablevault execute -db test_database_2 -t stories
tablevault_2.execute(table_name = 'stories')
#! tablevault execute -db test_database_2 -t llm_storage
tablevault_2.execute(table_name = 'llm_storage')


In [ ]:
# If we compare the two dataframes, note only the column's changed
#story_instance_ids = !tablevault list-instances -db test_database_2 -t stories
story_instance_ids = tablevault_2.list_instances(table_name = 'stories')

#code_instance_ids = !tablevault list-instances -db test_database_2 -t llm_storage
code_instance_ids = tablevault_2.list_instances(table_name = 'llm_storage')

for s_instance_id, c_instance_id,  in zip(story_instance_ids, code_instance_ids):
    df_path = f'test_database_2/stories/{s_instance_id}/table.csv'
    df = pd.read_csv(df_path)
    display(df)
    df_path = f'test_database_2/llm_storage/{c_instance_id}/table.csv'
    df = pd.read_csv(df_path)
    display(df)
    


In [ ]:
# IMPORTANT: Cleanup
delete_story()

# Delete Instance and Delete Table

Deletes are simple but need to be executed with the command lines, so that the metadata is correctly updated. By default nothing is every deleted.

In [ ]:
# Delete table instance 
#! tablevault delete-instance -db test_database -t stories -id {story_instance_id}
tablevault.delete_instance(instance_id = story_instance_id, table_name = 'stories')

In [ ]:
# Delete table
#! tablevault delete-table -db test_database -t llm_storage
tablevault.delete_table(table_name = 'llm_storage')

In [ ]:
# Show new folder and metadata
print_directory_tree(root_dir = 'test_database', indent=" ", files = False)

# Restarts

Not included in demo because requires mid-process stopping, but tested.

Restarts are executed when setting the restart flag when creating a new DataVault object.




# IMPORTANT - Cleanup
Cleanup OpenAI Storage after tests. Note: this cleans up all openai files -> might destroy other experiments

In [ ]:
def clean_up_open_ai(key_file = "open_ai_key/key.txt"):
    import openai
    from tqdm import tqdm
    with open(key_file, 'r') as f:
        secret = f.read()
        os.environ["OPENAI_API_KEY"] = secret
    client = openai.OpenAI()
    files = list(client.files.list())
    vector_stores = list(client.beta.vector_stores.list())
    my_assistants = list(client.beta.assistants.list())
    for store in tqdm(vector_stores):
        try:
          client.beta.vector_stores.delete(
            vector_store_id=store.id
          )
        except:
            pass
    for f in tqdm(files):
        try:
          client.files.delete(
            file_id=f.id
          )
        except:
          pass
    
    for assistant in tqdm(my_assistants):
        try:
            client.beta.assistants.delete(assistant.id)
        except:
            pass
    
    print(client.beta.vector_stores.list())
    print(client.files.list())
    print(client.beta.assistants.list())

In [ ]:
clean_up_open_ai()
shutil.rmtree('test_database')
shutil.rmtree('test_database_2')